In [1]:
import pandas as pd

In [ ]:
# Data from InsideAirBNB

df_calendar = pd.read_csv('Downloads/calendar.csv')
df_listing = pd.read_csv('Downloads/listings.csv')

df_calendar.columns = ['id', 'date', 'available', 'price']
df_calendar['date'] = pd.to_datetime(df_calendar['date'], format='%Y-%m-%d')

df_calendar = df_calendar.loc[df_calendar['available'] == 'f']

agg_duration = df_calendar.groupby(['id']).size().reset_index()
agg_duration.columns = ['id', 'duration']

df_final = df_listing.merge(agg_duration, on='id', how='left')
df_final.to_csv('Downloads/offre_durationbyID.csv', sep=";", index=False)

In [120]:
# Data from OpenData
df_loyer = pd.read_csv('Downloads/encadrement_loyers.csv')
df_loyer = df_loyer.loc[(df_loyer['ville'] == 'PARIS') & (df_loyer['annee'] >= 2017) & (df_loyer['meuble'] == 1)]
df_loyer_par_quartier = df_loyer.groupby(['nom_quartier'])['ref'].mean().reset_index()
df_loyer_par_quartier['nom_quartier'] = df_loyer_par_quartier['nom_quartier'].str.lower() \
                                                    .str.normalize('NFKD') \
                                                    .str.encode('ascii', errors='ignore') \
                                                    .str.decode('utf-8')
df_loyer_par_quartier['nom_quartier'] = df_loyer_par_quartier['nom_quartier'].replace([r'saint-',
                                                                                       r'sainte-',
                                                                                       r'faubourg-',
                                                                                       r'avoie',
                                                                                       r'^notre-dame-',
                                                                                       r'^parc-de-',
                                                                                       r' 15art',
                                                                                       r'^la '],
                                                                                      ['st-',
                                                                                       'ste-',
                                                                                       'fg-',
                                                                                       'avoye',
                                                                                       'nd-',
                                                                                       'parc-',
                                                                                       '',
                                                                                       ''
                                                                                       ], regex=True)

In [121]:
# Data from OpenStreetMap
df_quartier = pd.read_csv('Downloads/osm_paris_quartiers.csv', sep=";", usecols=["osm_id", "name",
                                     "addr.postcode", "admin_level",
                                     "alt_name", "boundary",
                                     "ref.INSEE", "short_name", "type",
                                     "wikidata", "wikipedia"])
df_quartier = df_quartier.loc[~df_quartier['short_name'].isnull()]
df_quartier['short_name'] = df_quartier['short_name'].str.lower() \
                                                    .str.normalize('NFKD') \
                                                    .str.encode('ascii', errors='ignore') \
                                                    .str.decode('utf-8')
df_quartier['short_name'] = df_quartier['short_name'].replace([r'^le ', r'^la ', r'les '], '', regex=True)
df_quartier['short_name'] = df_quartier['short_name'].replace(['enfants rouges',
                                                               r' des ',
                                                               'palais royal',
                                                               'vendome'],
                                                              ['enfants-rouges',
                                                               '-des-',
                                                               'palais-royal',
                                                               'place-vendome'], regex=True)

In [122]:
set(df_loyer_par_quartier['nom_quartier'].unique().tolist()) - set(df_quartier['short_name'].unique().tolist())

{'fg-du-roule', 'notre-dame', 'place-vendome', 'plaine de monceaux'}

In [125]:
df_loyer_par_quartier.loc[df_loyer_par_quartier['nom_quartier'].str.contains('moncea')]

,nom_quartier,ref
51,plaine de monceaux,27.10625


In [124]:
df_quartier.loc[df_quartier['short_name'].str.contains('vendo')]

,osm_id,name,addr.postcode,admin_level,alt_name,boundary,ref.INSEE,short_name,type,wikidata,wikipedia
10,2172743,Quartier Vendôme,75001,10,NaN,administrative,7510104,vendome,boundary,Q2720541,fr:Quartier de la Place-Vendôme
